## SKlearn NFM

#### Lets run SKLearn's NMF algorithm on a small 2d sample array

In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import NMF
from sklearn.metrics import mean_squared_error

In [6]:
print("Input matrix of N x N w/ values (1-10)")
def generateV(N=25):
    return 10 * np.random.random_sample((N, N))
X = generateV()
display(pd.DataFrame(X))

Input matrix of 100 x 100 w/ values (1-10)


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,7.469925,3.217139,9.678113,2.057477,9.928370,7.502708,4.235081,6.827450,6.875525,2.218488,...,8.208428,1.020215,7.709006,9.972137,5.719277,1.789128,9.870012,3.682720,0.354207,0.779295
1,0.492343,8.930575,8.248827,2.660873,4.482273,0.358903,5.959044,6.764288,7.667507,4.933397,...,5.917631,9.750772,5.716217,3.086872,3.031181,2.439550,1.676989,0.489930,5.016711,9.092692
2,8.403727,0.024528,2.356468,7.042988,0.035589,6.639081,7.627059,9.679604,5.773278,9.872118,...,8.290970,4.786296,9.242557,8.751690,5.811554,2.112398,8.937850,3.516568,1.810831,6.202491
3,8.809263,5.212518,5.456266,3.120250,7.502048,6.621667,5.400025,0.425679,0.963815,5.755994,...,7.890415,0.916301,3.059526,8.047528,8.282066,2.767486,9.777793,6.847187,8.704330,0.258649
4,2.091279,5.935401,0.967219,6.586931,0.031507,5.235649,4.886343,7.242988,5.027380,8.677378,...,9.639021,1.907452,0.612589,8.655389,6.431038,7.093082,6.559135,7.897795,0.352055,0.505713
5,6.237587,6.971097,9.514483,9.388969,0.875569,3.034860,7.852125,1.788060,5.913626,0.593757,...,0.729119,4.630053,1.698429,7.776649,3.758755,5.977210,7.046387,4.966371,3.928538,2.242835
6,9.776704,3.709425,6.086331,1.430915,7.185617,0.414546,1.058556,3.461595,1.076290,8.969618,...,7.606004,8.721357,3.390524,8.765202,5.623860,8.836263,4.576385,7.958496,1.708872,3.426277
7,4.859800,8.017076,6.343148,3.619045,3.400580,4.982871,9.722879,9.091746,3.971069,2.834799,...,0.008762,5.829153,5.300225,5.680492,4.159923,3.448068,2.348539,5.652769,2.420827,8.205122
8,5.339686,8.173200,9.312147,5.614151,6.723538,0.169439,7.203953,4.205249,8.091347,3.432280,...,4.328415,8.785987,3.525778,7.181246,5.855179,9.871863,4.566968,9.626024,6.309222,2.487707
9,9.069721,9.082622,7.541968,5.925287,6.401856,5.060040,2.237011,9.971606,4.400707,7.217998,...,4.040842,0.447974,9.363232,0.753175,8.243888,0.012217,9.111659,9.611889,8.180458,9.087577


In [7]:
N = X.shape[1]
v_outs = []
for i in range(1, N + 1): # inclusive range to N
    skl_nmf = NMF(n_components=i, init ='random', random_state=0)
    W = pd.DataFrame(skl_nmf.fit_transform(X))
    H = pd.DataFrame(skl_nmf.components_)
    V = np.dot(W, H)
    err = 0
    for k in range(len(X)):
        err += mean_squared_error(V[k], X[k])
    v_outs.append(err / len(X))

#### Here is our mean_squared_error for modifying n_components in SKLearn's NMF Algorithm

In [8]:
display(pd.DataFrame(v_outs).describe())
pd.Series(v_outs).plot()

,0
count,25.000000
mean,2.066458
std,2.247083
min,0.047530
25%,0.300362
50%,1.146734
75%,3.105280
max,7.719012


In [9]:
print ("Computed V from W and H")
display(pd.DataFrame(np.dot(W, H)))
print("\nActual input V")
display(pd.DataFrame(X))

Computed V from W and H


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,7.599581,3.124230,9.531321,2.006625,9.762876,7.751311,4.379445,6.914715,6.551132,2.219955,...,8.266895,1.287954,7.855189,10.051423,5.604318,1.781446,9.732807,3.925115,0.731254,0.868109
1,0.487780,8.995906,8.214020,2.632373,4.537937,0.849127,5.996977,6.693850,7.513186,4.667673,...,5.896618,9.876739,5.849328,3.158354,3.064339,2.196780,1.608346,1.008910,5.095492,9.013587
2,8.235780,0.171150,2.546910,6.763453,0.327525,6.841386,7.399463,9.945637,5.790227,9.815381,...,8.278114,4.804559,9.311590,8.785273,5.828683,2.579164,8.969542,3.726516,2.098824,6.045075
3,8.815010,5.044425,5.610473,3.288554,7.630579,6.558010,5.321667,0.633587,1.461299,5.706207,...,7.595870,1.171539,3.129232,7.940238,8.251178,3.025522,9.803486,6.809839,8.574366,0.672312
4,2.397356,5.979110,1.037506,6.877347,0.423636,5.155650,5.111934,7.145333,5.129452,8.723727,...,9.636189,2.101993,1.010850,8.578697,6.324746,6.642880,6.524870,7.909159,0.789422,0.578610
5,6.026968,6.900374,9.654371,9.236243,0.947889,2.805897,7.828027,2.060618,5.943186,1.077312,...,1.244792,4.510154,1.653362,7.691272,3.783254,6.048587,7.148545,5.250280,3.581022,2.395873
6,9.607727,3.790074,6.183698,1.259930,7.337186,1.116999,1.163699,3.367224,1.210742,8.819928,...,7.745069,8.681860,3.350712,8.634226,5.736045,8.685460,4.684568,8.125284,1.813607,3.545368
7,4.801731,7.929319,6.385424,3.662514,3.613084,5.033666,9.440672,9.057852,4.211198,2.714784,...,0.380721,5.710193,5.153117,5.803260,3.958992,3.973899,2.361077,5.645302,2.691704,8.311295
8,5.355928,8.377798,9.097915,5.735188,6.571365,0.944507,7.223937,4.006598,7.929909,3.573963,...,4.543632,9.079013,3.635201,7.336189,5.958931,9.613374,4.390948,9.150974,6.341077,2.202203
9,9.168977,9.048731,7.537657,6.131569,6.210634,5.087716,2.815360,9.754523,4.237328,7.539433,...,4.248673,0.691294,9.185598,0.727114,8.412520,0.411136,9.134896,9.307970,7.837895,9.211985



Actual input V


,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,7.469925,3.217139,9.678113,2.057477,9.928370,7.502708,4.235081,6.827450,6.875525,2.218488,...,8.208428,1.020215,7.709006,9.972137,5.719277,1.789128,9.870012,3.682720,0.354207,0.779295
1,0.492343,8.930575,8.248827,2.660873,4.482273,0.358903,5.959044,6.764288,7.667507,4.933397,...,5.917631,9.750772,5.716217,3.086872,3.031181,2.439550,1.676989,0.489930,5.016711,9.092692
2,8.403727,0.024528,2.356468,7.042988,0.035589,6.639081,7.627059,9.679604,5.773278,9.872118,...,8.290970,4.786296,9.242557,8.751690,5.811554,2.112398,8.937850,3.516568,1.810831,6.202491
3,8.809263,5.212518,5.456266,3.120250,7.502048,6.621667,5.400025,0.425679,0.963815,5.755994,...,7.890415,0.916301,3.059526,8.047528,8.282066,2.767486,9.777793,6.847187,8.704330,0.258649
4,2.091279,5.935401,0.967219,6.586931,0.031507,5.235649,4.886343,7.242988,5.027380,8.677378,...,9.639021,1.907452,0.612589,8.655389,6.431038,7.093082,6.559135,7.897795,0.352055,0.505713
5,6.237587,6.971097,9.514483,9.388969,0.875569,3.034860,7.852125,1.788060,5.913626,0.593757,...,0.729119,4.630053,1.698429,7.776649,3.758755,5.977210,7.046387,4.966371,3.928538,2.242835
6,9.776704,3.709425,6.086331,1.430915,7.185617,0.414546,1.058556,3.461595,1.076290,8.969618,...,7.606004,8.721357,3.390524,8.765202,5.623860,8.836263,4.576385,7.958496,1.708872,3.426277
7,4.859800,8.017076,6.343148,3.619045,3.400580,4.982871,9.722879,9.091746,3.971069,2.834799,...,0.008762,5.829153,5.300225,5.680492,4.159923,3.448068,2.348539,5.652769,2.420827,8.205122
8,5.339686,8.173200,9.312147,5.614151,6.723538,0.169439,7.203953,4.205249,8.091347,3.432280,...,4.328415,8.785987,3.525778,7.181246,5.855179,9.871863,4.566968,9.626024,6.309222,2.487707
9,9.069721,9.082622,7.541968,5.925287,6.401856,5.060040,2.237011,9.971606,4.400707,7.217998,...,4.040842,0.447974,9.363232,0.753175,8.243888,0.012217,9.111659,9.611889,8.180458,9.087577


## Testing Accuracy with NMF:

## Comparing to PCA

<img src="compare_pca.jpg" alt="Drawing" style="width: 1000px;"/>

In [26]:
from sklearn.decomposition import PCA
from scipy.stats import pearsonr

V = generateV()

We need some way to compare nmf and pca at dimensions higher when reducing to dimensions higher than two. One way to do this is to use a pearson correlation test between pairs of basis vectors from NMF and PCA respectively.  

Below highlights the usgage of scipys pearsonr function which we use to calculate Pearson's correlation coefficient from our nmf and pca basis vectors.

In [35]:
x = [1,2,3,4,5,6,7,8,9,10]
y = [2,4,6,8,10,12,14,16,18,20]
print("Pearson's correlation coefficient = {}".format(((pearsonr(x,y)[0] + pearsonr(y,x) )/ 2)[0]))

Pearson's correlation coefficient = 0.9999999999999998


In [32]:
def correlate(nmf_i, pca_i):
    return pearsonr(nmf_i, pca_i)[0]

Averaging all possible pairs Pearson correlation coefficient

In [ ]:
# TODO need NMF model function
def compareNMFtoPCA(V):
    Vn = np.array(V)
    F = len(V) # feature count
    pair_matrices = []
    for i in range(F,1,-1):
        pca_model = PCA(n_components=i).fit(Vn)
        p = pca_model.components_.transpose()
        pairs = [i][i]
        # w, h = NMF(V, i)
        # h = h.transpose()
        for nmf_i in range(i):
            for pca_i in range(i):
                # pairs[nmf_i][pca_i] = correlate(h[nmf_i], p[pca_i])
        pair_matrices.append(pairs)
        
compareNMFtoPCA(V) # outputs list of pair matrices containing pearson correlation values for each [ith dimension][nmf_ith basis vector][pca_ith basis vector]
# need to run analysis on above. maybe avg pair correlation values at each iteration?

## Recursive NMF Dimensionality Reduction? 
Do we lose more info in aproximating each step or losing the dimensions?

In [ ]:
def recur_reduce_nmf(V, target):
    # w, h = NMF(V, len(V) - 1)
    if(len(V) == target + 1):
        return w, h
    
    new_V = np.dot(w, h)
    return recur_reduce_nmf(new_V, target)